# neuralcoref 사용법

In [1]:
import spacy
nlp = spacy.load('en')

import neuralcoref
neuralcoref.add_to_pipe(nlp)

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [2]:
import spacy
nlp = spacy.load('en')

import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

doc = nlp(u'My sister has a dog. She loves him.')

print(doc._.has_coref)
print(doc._.coref_clusters)

True
[My sister: [My sister, She], a dog: [a dog, him]]


In [3]:
print(f'doc은\n{doc}')
print(doc._.coref_resolved)
print(doc._.coref_scores)
print('')
span = doc[-7:-5]
print(f'span은\n{span}')

print(span._.is_coref)
print(span._.coref_cluster)
print(span._.coref_scores)
print('')
token = doc[-10]
print(f'token은\n{token}')

print(token._.in_coref)
print(token._.coref_clusters)

doc은
My sister has a dog. She loves him.
My sister has a dog. My sister loves a dog.
{My sister: {My sister: 1.3110305070877075}, a dog: {a dog: 1.804752230644226, My sister: -1.6715972423553467}, She: {She: -0.10834205150604248, My sister: 8.058426856994629, a dog: -1.0625176429748535}, him: {him: -1.870743989944458, My sister: 3.1147186756134033, a dog: 4.356405258178711, She: -3.1379528045654297}}

span은
a dog
True
a dog: [a dog, him]
{a dog: 1.804752230644226, My sister: -1.6715972423553467}

token은
My
True
[My sister: [My sister, She]]


In [4]:
import spacy
import neuralcoref

nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)
doc = nlp(u'Deepika has a dog. She loves him. The movie star has always been fond of animals')
print(doc._.coref_clusters)
print(doc._.coref_resolved)

[Deepika: [Deepika, She, him, The movie star]]
Deepika has a dog. Deepika loves Deepika. Deepika has always been fond of animals


In [5]:
nlp.remove_pipe("neuralcoref")
neuralcoref.add_to_pipe(nlp, conv_dict={'Deepika':['woman','actress']})

doc = nlp(u'Deepika has a dog. She loves him. The movies star has always been fond of animals')
doc._.coref_clusters

[Deepika: [Deepika, She, The movies star], a dog: [a dog, him]]

In [6]:
doc._.coref_resolved

'Deepika has a dog. Deepika loves a dog. Deepika has always been fond of animals'

# Coreference Resolution

In [7]:
# spacy load하기
nlp = spacy.load('en')

# spacy pipe에 neural coref 추가하기
neuralcoref.add_to_pipe(nlp)

def coreference_resolution(text):
    '''
    주어진 텍스트에 대해 coreference resolution을 수행하고
    그 결과에 맞게 대표 단어들로 대체시켜 다시 text를 반환하는 함수
    '''
    nlp = spacy.load('en')
    neuralcoref.add_to_pipe(nlp)
    doc = nlp(text)
    return doc._.coref_resolved

text = '''
Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX.
He is also early investor, CEO, and product architect of Tesla, Inc.
'''

print(text)
print('-'*50)
print(coreference_resolution(text))
    


Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX.
He is also early investor, CEO, and product architect of Tesla, Inc.

--------------------------------------------------

Elon Musk is a business magnate, industrial designer, and engineer.

Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.

Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.



# Named Entity Linking

In [8]:
import spacy

nlp = spacy.load("en")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion.")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [9]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [10]:
def NER(text):
    '''
    주어진 text에 대해 named entity recognition을 수행하고
    (entity 이름, (entity의 시작 인덱스, entity의 끝 인덱스), entity 종류)의 튜플들이
    담긴 리스트를 반환하는 함수
    '''
    nlp = spacy.load('en')
    doc = nlp(text)
    ent_lst = [(ent.text, (ent.start_char, ent.end_char), ent.label_) for ent in doc.ents]
    
    return ent_lst

In [11]:
text_ex = "Apple is looking at buying U.K. startup for $1 billion"
NER(text_ex)

[('Apple', (0, 5), 'ORG'),
 ('U.K.', (27, 31), 'GPE'),
 ('$1 billion', (44, 54), 'MONEY')]

In [12]:
print(len(NER(text_ex)))
print(NER(text_ex)[0])

3
('Apple', (0, 5), 'ORG')


# Relation Extraction

In [13]:
import opennre

In [14]:
model = opennre.get_model('wiki80_cnn_softmax')

2021-02-25 10:28:03,688 - root - INFO - Initializing word embedding with word2vec.


In [15]:
t = 'Bill Gates founded Microsoft.'
doc = nlp(t)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Bill Gates 0 10 PERSON
Microsoft 19 28 ORG


In [16]:
t[0:10]

'Bill Gates'

In [17]:
t[19:28]

'Microsoft'

In [18]:
model.infer({'text':'Bill Gates founded Microsoft.', 'h':{'pos': (0, 10)}, 't': {'pos': (19, 28)}})

('field of work', 0.32416045665740967)

In [19]:
t = 'Steve Jobs founded Apple.'
t.index('Apple')

19

In [20]:
t = 'President Ford granted a pardon to President Nixon.'
doc = nlp(t)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Ford 10 14 PERSON
Nixon 45 50 PERSON


In [21]:
model.infer({'text':t, 'h':{'pos':(10,14)}, 't': {'pos':(45,50)}})

('head of government', 0.5379694700241089)

In [22]:
type(model.infer({'text':t, 'h':{'pos':(10,14)}, 't': {'pos':(45,50)}}))

tuple

In [23]:
a = (10, 14)
b = (45, 50)
model.infer({'text':t, 'h':{'pos':a}, 't': {'pos':b}})

('head of government', 0.5379694700241089)

In [24]:
def RE(text, head_entity_idx, tail_entity_idx, RE_model):
    '''
    주어진 text에 대해 head entity와 tail entity의 위치를 알고 있을 때
    relation extraction을 수행하고 entity 간의 관계를 반환하는 함수
    '''
    model = opennre.get_model(RE_model)
    result = model.infer({'text': text, 'h':{'pos': head_entity_idx}, 't': {'pos': tail_entity_idx}})
    
    return result

In [25]:
text = 'President Ford granted a pardon to President Nixon.'
print(NER(text))

[('Ford', (10, 14), 'PERSON'), ('Nixon', (45, 50), 'PERSON')]


In [26]:
h_idx = NER(text)[0][1]
t_idx = NER(text)[1][1]

RE_model = 'wiki80_cnn_softmax'

print(RE(text, h_idx, t_idx, RE_model))

2021-02-25 10:28:05,493 - root - INFO - Initializing word embedding with word2vec.


('head of government', 0.5379694700241089)


In [27]:
text = "Apple is looking at buying U.K. startup for $1 billion"
print(NER(text_ex))

[('Apple', (0, 5), 'ORG'), ('U.K.', (27, 31), 'GPE'), ('$1 billion', (44, 54), 'MONEY')]


In [28]:
from itertools import permutations
a = NER(text_ex)
candidates = list(permutations(a,2))
candidates

[(('Apple', (0, 5), 'ORG'), ('U.K.', (27, 31), 'GPE')),
 (('Apple', (0, 5), 'ORG'), ('$1 billion', (44, 54), 'MONEY')),
 (('U.K.', (27, 31), 'GPE'), ('Apple', (0, 5), 'ORG')),
 (('U.K.', (27, 31), 'GPE'), ('$1 billion', (44, 54), 'MONEY')),
 (('$1 billion', (44, 54), 'MONEY'), ('Apple', (0, 5), 'ORG')),
 (('$1 billion', (44, 54), 'MONEY'), ('U.K.', (27, 31), 'GPE'))]

In [29]:
for cand in candidates:
    h, t = cand
    h_text, t_text = h[0], t[0]
    h_idx, t_idx = h[1], t[1]
    h_label_, t_label_ = h[2], t[2]
    r = RE(text, h_idx, t_idx, RE_model)
    r_text, r_prob = r[0], r[1]
    print((h_text, r_text, t_text, r_prob))

2021-02-25 10:28:06,626 - root - INFO - Initializing word embedding with word2vec.


('Apple', 'subsidiary', 'U.K.', 0.36490923166275024)


2021-02-25 10:28:07,259 - root - INFO - Initializing word embedding with word2vec.


('Apple', 'subsidiary', '$1 billion', 0.3249340057373047)


2021-02-25 10:28:07,890 - root - INFO - Initializing word embedding with word2vec.


...

('U.K.', 'subsidiary', '$1 billion', 0.6145082116127014)


2021-02-25 10:28:09,169 - root - INFO - Initializing word embedding with word2vec.


('$1 billion', 'manufacturer', 'Apple', 0.39467406272888184)


2021-02-25 10:28:09,799 - root - INFO - Initializing word embedding with word2vec.


('$1 billion', 'owned by', 'U.K.', 0.4790058434009552)


# Pipeline
input text -> coreference resolution -> named entity recognition -> relation extraction

In [30]:
import spacy
import neuralcoref
import opennre
from itertools import permutations
from nltk.tokenize import sent_tokenize

RE_model = 'wiki80_cnn_softmax'

def txt2triple(text):
    '''
    주어진 텍스트에 대해 coreference resolution을 수행하고
    대체시킨 text에서 named entity recognition을 통해 찾은
    entities에 대해 relation을 추출하는 함수
    '''
    # coreference resolution
    text = coreference_resolution(text)
    
    # named entity recognition
    ent_info = NER(text)
    if len(ent_info) >= 2:
        candidates = list(permutations(ent_info, 2))
    
        # relation extraction
        triple_list = []
        for cand in candidates:
            h, t = cand
            
            h_text, t_text = h[0], t[0]
            h_idx, t_idx = h[1], t[1]
            h_label_, t_label_ = h[2], t[2]
            
            r = RE(text, h_idx, t_idx, RE_model)
            r_text, r_prob = r[0], r[1]
            
            triple = ((h_text, r_text, t_text), r_prob)
            triple_list.append(triple)
            
        return triple_list

In [31]:
text = '''
Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX.
He is also early investor, CEO, and product architect of Tesla, Inc.
'''
text = coreference_resolution(text)
print(text)
[x.strip() for x in sent_tokenize(text)]


Elon Musk is a business magnate, industrial designer, and engineer.

Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.

Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.



['Elon Musk is a business magnate, industrial designer, and engineer.',
 'Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.',
 'Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.']

In [32]:
from nltk.tokenize import sent_tokenize
sent_tokenize(text)

['\nElon Musk is a business magnate, industrial designer, and engineer.',
 'Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.',
 'Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.']

In [33]:
t = "Apple is looking at buying U.K. startup for $1 billion"
txt2triple(t)

2021-02-25 10:28:11,365 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:11,997 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:12,646 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:13,285 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:13,923 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:14,577 - root - INFO - Initializing word embedding with word2vec.


[(('Apple', 'subsidiary', 'U.K.'), 0.36490923166275024),
 (('Apple', 'subsidiary', '$1 billion'), 0.3249340057373047),
 (('U.K.', 'subsidiary', 'Apple'), 0.826092004776001),
 (('U.K.', 'subsidiary', '$1 billion'), 0.6145082116127014),
 (('$1 billion', 'manufacturer', 'Apple'), 0.39467406272888184),
 (('$1 billion', 'owned by', 'U.K.'), 0.4790058434009552)]

In [34]:
text = '''
Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX.
He is also early investor, CEO, and product architect of Tesla, Inc.
'''
print(coreference_resolution(text))


Elon Musk is a business magnate, industrial designer, and engineer.

Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.

Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.



In [35]:
txt2triple(text)

2021-02-25 10:28:16,045 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:16,682 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:17,336 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:17,979 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:18,612 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:19,253 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:19,888 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:20,535 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:21,187 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:21,820 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:22,458 - root - INFO - Initializing word embedding with word2vec.
2021-02-25 10:28:23,083 - root - INFO - Initializing word embedding with word2vec.
2021

[(('Elon Musk', 'child', 'Elon Musk'), 0.2749059498310089),
 (('Elon Musk', 'notable work', 'SpaceX.'), 0.2706102728843689),
 (('Elon Musk', 'child', 'Elon Musk'), 0.21883255243301392),
 (('Elon Musk', 'manufacturer', 'Tesla, Inc.'), 0.6464232802391052),
 (('Elon Musk', 'architect', 'Elon Musk'), 0.23258407413959503),
 (('Elon Musk', 'owned by', 'SpaceX.'), 0.1680341511964798),
 (('Elon Musk', 'architect', 'Elon Musk'), 0.31130483746528625),
 (('Elon Musk', 'manufacturer', 'Tesla, Inc.'), 0.44388604164123535),
 (('SpaceX.', 'architect', 'Elon Musk'), 0.2609061002731323),
 (('SpaceX.', 'architect', 'Elon Musk'), 0.29049351811408997),
 (('SpaceX.', 'architect', 'Elon Musk'), 0.28137004375457764),
 (('SpaceX.', 'manufacturer', 'Tesla, Inc.'), 0.5890234112739563),
 (('Elon Musk', 'architect', 'Elon Musk'), 0.2562868595123291),
 (('Elon Musk', 'architect', 'Elon Musk'), 0.2746952176094055),
 (('Elon Musk', 'owned by', 'SpaceX.'), 0.2741602957248688),
 (('Elon Musk', 'manufacturer', 'Tesla, 

In [36]:
import spacy
import neuralcoref
import opennre
from itertools import permutations
from nltk.tokenize import sent_tokenize
from pandas import DataFrame

def coreference_resolution(text):
    '''
    주어진 텍스트에 대해 coreference resolution을 수행하고
    그 결과에 맞게 대표 단어들로 대체시켜 다시 text를 반환하는 함수
    '''
    nlp = spacy.load('en')
    neuralcoref.add_to_pipe(nlp)
    doc = nlp(text)
    return doc._.coref_resolved

def NER(text):
    '''
    주어진 text에 대해 named entity recognition을 수행하고
    (entity 이름, (entity의 시작 인덱스, entity의 끝 인덱스), entity 종류)의 튜플들이
    담긴 리스트를 반환하는 함수
    '''
    nlp = spacy.load('en')
    doc = nlp(text)
    ent_lst = [(ent.text, (ent.start_char, ent.end_char), ent.label_) for ent in doc.ents]
    
    return ent_lst

def RE(text, head_entity_idx, tail_entity_idx, RE_model):
    '''
    주어진 text에 대해 head entity와 tail entity의 위치를 알고 있을 때
    relation extraction을 수행하고 entity 간의 관계를 반환하는 함수
    '''
    model = opennre.get_model(RE_model)
    result = model.infer({'text': text, 'h':{'pos': head_entity_idx}, 't': {'pos': tail_entity_idx}})
    
    return result

def txt2triple(text, RE_model, relation_threshold=0.8):
    '''
    주어진 텍스트에 대해 coreference resolution을 수행하고
    대체시킨 text에서 named entity recognition을 통해 찾은
    entities에 대해 relation을 추출하는 함수
    '''
    # coreference resolution
    text = coreference_resolution(text)
    
    sentences = [x for x in sent_tokenize(text)]
#     print(sentences)
    triple_list = []
    
    for sent in sentences:
        # named entity recognition
        ent_info = NER(sent)
#         print(ent_info)
        if len(ent_info) >= 2:
            candidates = list(permutations(ent_info, 2))
    
            # relation extraction
            for cand in candidates:
                h, t = cand
            
                h_text, t_text = h[0], t[0]
                h_idx, t_idx = h[1], t[1]
                h_label_, t_label_ = h[2], t[2]
            
                r = RE(text, h_idx, t_idx, RE_model)
                r_text, r_prob = r[0], r[1]
                
                if h_text != t_text and r_prob > relation_threshold:
                    triple = (sent, (h_text, r_text, t_text), r_prob)
                    triple_list.append(triple)
                    
    df = DataFrame(triple_list, columns=['original text','triple','confidence level'])
    df.sort_values(by=['confidence level'], ascending=False, inplace=True)
    
#     return triple_list if triple_list else None
    return df if not df.empty else None


In [38]:
RE_model = 'wiki80_bert_softmax'

text = '''
Elon Musk is a business magnate, industrial designer, and engineer.
He is the founder, CEO, CTO, and chief designer of SpaceX. 
He is also early investor, CEO, and product architect of Tesla, Inc.
'''
txt2triple(text)

2021-02-25 10:28:29,174 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:32,581 - root - INFO - Loading BERT pre-trained checkpoint.


,original text,triple,confidence level
0,"Elon Musk is also early investor, CEO, and pro...","(Elon Musk, spouse, Tesla, Inc.)",0.979075
1,"Elon Musk is also early investor, CEO, and pro...","(Tesla, Inc., spouse, Elon Musk)",0.943041


In [39]:
txt2triple('Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.')

2021-02-25 10:28:36,362 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:39,687 - root - INFO - Loading BERT pre-trained checkpoint.


In [40]:
txt2triple('Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.')

In [41]:
text = '''
Elon Musk is a business magnate, industrial designer, and engineer. He is the founder, CEO, CTO, and chief designer of SpaceX. He is also early investor, CEO, and product architect of Tesla, Inc. He is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. He transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. He went on co-founding a web software company Zip2 with his brother Kimbal Musk.
'''

In [42]:
txt2triple(text)

2021-02-25 10:28:44,477 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:47,498 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:50,590 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:53,680 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:56,824 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:28:59,996 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:03,178 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:06,277 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:09,388 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:12,688 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:16,089 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:18,895 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:29:21,702 - root - INFO - Loading BERT pre-trained checkpoint.

,original text,triple,confidence level
36,Elon Musk briefly attended the University of P...,"(17, has part, Queen's University)",0.995863
56,Elon Musk went on co-founding a web software c...,"(Kimbal Musk, has part, Elon Musk)",0.995241
42,Elon Musk transferred to Queen's University tw...,"(two years later, has part, Queen's University)",0.995171
39,Elon Musk briefly attended the University of P...,"(Queen's University, has part, 17)",0.994733
46,Elon Musk moved to California in 1995 to atten...,"(California, has part, Elon Musk)",0.993842
33,Elon Musk briefly attended the University of P...,"(17, has part, Elon Musk)",0.993656
14,"A centibillionaire, \nElon Musk became the ric...","($185 billion, has part, Elon Musk)",0.992863
40,Elon Musk transferred to Queen's University tw...,"(Queen's University, has part, two years later)",0.992819
10,"A centibillionaire, \nElon Musk became the ric...","(Elon Musk, has part, $185 billion)",0.992446
7,"Elon Musk is also early investor, CEO, and pro...","(Neuralink, has part, Elon Musk)",0.991472


In [43]:
print(txt2triple('Elon Musk is a business magnate, industrial designer, and engineer.'))
print(txt2triple('Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX.'))
print(txt2triple('Elon Musk is also early investor, CEO, and product architect of Tesla, Inc.'))
print(txt2triple('Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink.'))
print(txt2triple('A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos.'))
print(txt2triple('Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa.'))
print(txt2triple("Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University."))
print(txt2triple("Elon Musk transferred to the University of Pennsylvania two years later, where he received dual bachelor's degrees in economics and physics."))
print(txt2triple("Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career."))
print(txt2triple("Elon Musk went on co-founding a web software company Zip2 with his brother Kimbal Musk."))



None
None


2021-02-25 10:33:40,020 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:42,615 - root - INFO - Loading BERT pre-trained checkpoint.


None


2021-02-25 10:33:45,636 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:48,318 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:50,887 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:53,493 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:56,092 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:33:58,818 - root - INFO - Loading BERT pre-trained checkpoint.


                                       original text  \
2  Elon Musk is also the founder of The Boring Co...   
4  Elon Musk is also the founder of The Boring Co...   
0  Elon Musk is also the founder of The Boring Co...   
3  Elon Musk is also the founder of The Boring Co...   
5  Elon Musk is also the founder of The Boring Co...   
1  Elon Musk is also the founder of The Boring Co...   

                                        triple  confidence level  
2    (The Boring Company, owned by, Elon Musk)          0.992970  
4             (Neuralink, owned by, Elon Musk)          0.986557  
0    (Elon Musk, owned by, The Boring Company)          0.956924  
3  (The Boring Company, subsidiary, Neuralink)          0.954756  
5  (Neuralink, subsidiary, The Boring Company)          0.873525  
1             (Elon Musk, owned by, Neuralink)          0.847245  


...

                                       original text  \
5  Elon Musk transferred to the University of Pen...   
0  Elon Musk transferred to the University of Pen...   
6  Elon Musk transferred to the University of Pen...   
3  Elon Musk transferred to the University of Pen...   
4  Elon Musk transferred to the University of Pen...   
1  Elon Musk transferred to the University of Pen...   
2  Elon Musk transferred to the University of Pen...   

                                              triple  confidence level  
5  (Elon Musk, work location, the University of P...          0.954321  
0  (Elon Musk, work location, the University of P...          0.950841  
6              (Elon Musk, sibling, two years later)          0.905605  
3  (two years later, subsidiary, the University o...          0.903049  
4              (two years later, sibling, Elon Musk)          0.899068  
1  (the University of Pennsylvania, subsidiary, t...          0.850591  
2  (the University of Pennsylvania, work

2021-02-25 10:36:54,039 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:36:56,615 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:36:59,279 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:01,974 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:04,685 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:07,318 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:09,916 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:12,508 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:15,097 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:17,718 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:20,475 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:23,106 - root - INFO - Loading BERT pre-trained checkpoint.


                                       original text  \
0  Elon Musk moved to California in 1995 to atten...   
2  Elon Musk moved to California in 1995 to atten...   
4  Elon Musk moved to California in 1995 to atten...   
1  Elon Musk moved to California in 1995 to atten...   
7  Elon Musk moved to California in 1995 to atten...   
5  Elon Musk moved to California in 1995 to atten...   
3  Elon Musk moved to California in 1995 to atten...   
6  Elon Musk moved to California in 1995 to atten...   

                                 triple  confidence level  
0    (Elon Musk, residence, California)          0.997232  
2    (California, residence, Elon Musk)          0.996242  
4        (1995, participant, Elon Musk)          0.964369  
1        (Elon Musk, participant, 1995)          0.952421  
7  (Stanford University, follows, 1995)          0.944061  
5          (1995, location, California)          0.917068  
3          (California, location, 1995)          0.913099  
6  (1995, follo

2021-02-25 10:37:26,140 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:28,707 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:31,300 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:33,983 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:36,586 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:39,191 - root - INFO - Loading BERT pre-trained checkpoint.


                                       original text  \
2  Elon Musk went on co-founding a web software c...   
0  Elon Musk went on co-founding a web software c...   
3  Elon Musk went on co-founding a web software c...   
1  Elon Musk went on co-founding a web software c...   

                              triple  confidence level  
2  (Kimbal Musk, sibling, Elon Musk)          0.997942  
0  (Elon Musk, sibling, Kimbal Musk)          0.997933  
3  (Kimbal Musk, sibling, Elon Musk)          0.997929  
1  (Elon Musk, sibling, Kimbal Musk)          0.997869  


In [44]:
txt2triple("The Marvel Cinematic Universe (MCU) is an American media franchise and shared universe centered on a series of superhero films, independently produced by Marvel Studios and based on characters that appear in American comic books published by Marvel Comics. The franchise includes comic books, short films, television series, and digital series. The shared universe, much like the original Marvel Universe in comic books, was established by crossing over common plot elements, settings, cast, and characters.")

2021-02-25 10:37:42,189 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:44,716 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:47,416 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:50,079 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:52,677 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:55,276 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:37:57,821 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:00,505 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:03,113 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:05,708 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:08,377 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:10,917 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:13,571 - root - INFO - Loading BERT pre-trained checkpoint.

,original text,triple,confidence level
4,The Marvel Cinematic Universe (MCU) is an Amer...,"(Marvel Comics, has part, MCU)",0.980686
0,The Marvel Cinematic Universe (MCU) is an Amer...,"(MCU, country of origin, American)",0.931788
1,The Marvel Cinematic Universe (MCU) is an Amer...,"(MCU, part of, Marvel Comics)",0.885659
2,The Marvel Cinematic Universe (MCU) is an Amer...,"(American, part of, MCU)",0.863393
5,The Marvel Cinematic Universe (MCU) is an Amer...,"(Marvel Comics, subsidiary, Marvel Studios)",0.850024
3,The Marvel Cinematic Universe (MCU) is an Amer...,"(Marvel Studios, subsidiary, Marvel Comics)",0.829021


In [45]:
txt2triple("""The first MCU film is Iron Man (2008), which began the films of Phase One culminating in the crossover film The Avengers (2012). Phase Two began with Iron Man 3 (2013) and concluded with Ant-Man (2015). Phase Three began with Captain America: Civil War (2016) and concluded with Spider-Man: Far From Home (2019). The first three phases in the franchise are collectively known as "The Infinity Saga". The films of Phase Four will begin with Black Widow (2021) and are set to conclude with Guardians of the Galaxy Vol. 3 (2023).""")

2021-02-25 10:38:36,029 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:38,689 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:41,383 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:43,989 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:46,617 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:49,289 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:51,880 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:54,584 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:57,211 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:38:59,814 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:39:02,483 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:39:05,215 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:39:07,820 - root - INFO - Loading BERT pre-trained checkpoint.

,original text,triple,confidence level
17,Phase Two began with Iron Man 3 (2013) and con...,"(2013, has part, Ant-Man)",0.994713
3,"The first MCU film is Iron Man (2008), which b...","(MCU, instance of, One)",0.993919
10,"The first MCU film is Iron Man (2008), which b...","(One, instance of, MCU)",0.993033
12,"The first MCU film is Iron Man (2008), which b...","(One, follows, 2008)",0.992197
40,The first three phases in the franchise are co...,"(three, has part, The Infinity Saga)",0.991550
18,Phase Two began with Iron Man 3 (2013) and con...,"(Ant-Man, has part, 2013)",0.991530
8,"The first MCU film is Iron Man (2008), which b...","(2008, follows, One)",0.989143
2,"The first MCU film is Iron Man (2008), which b...","(MCU, instance of, 2008)",0.987924
14,"The first MCU film is Iron Man (2008), which b...","(2012, instance of, MCU)",0.987523
5,"The first MCU film is Iron Man (2008), which b...","(Iron Man, instance of, MCU)",0.983998


In [46]:
txt2triple('Steve Jobs is the founder of Apple.')

2021-02-25 10:42:44,553 - root - INFO - Loading BERT pre-trained checkpoint.
2021-02-25 10:42:47,176 - root - INFO - Loading BERT pre-trained checkpoint.


,original text,triple,confidence level
1,Steve Jobs is the founder of Apple.,"(Apple, owned by, Steve Jobs)",0.970344
0,Steve Jobs is the founder of Apple.,"(Steve Jobs, owned by, Apple)",0.878194
